In [8]:
from huggingface_hub import snapshot_download
from insightface.app import FaceAnalysis
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sqlite3
import os

In [9]:
snapshot_download(
    "fal/AuraFace-v1",
    local_dir="models/auraface",
)

Returning existing local_dir `models\auraface` as remote repo cannot be accessed in `snapshot_download` ((ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: aa899c7c-3ae9-4498-af78-14d4427da6f6)')).


'C:\\Users\\dhair\\Documents\\VS-Code-Practice-Files-main\\ubiq_project\\models\\auraface'

In [10]:
face_app = FaceAnalysis(
    name="auraface",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],
    root=".",
)
face_app.prepare(ctx_id=0)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\glintr100.onnx recognition ['None', 3, 112, 112] 127.5 127.5
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: .\models\auraface\scrfd_10g_bnkps.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (640, 640)


In [11]:
def update_student_embeddings(roll_no, folder):
    image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith((".jpg",".png",".jpeg"))]
    image_files.sort()

    embeddings = []
    for img_path in image_files[:3]:
        img = cv2.imread(img_path)
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = face_app.get(img_rgb)
        if len(faces) > 0:
            emb = faces[0].normed_embedding.astype(np.float32).tobytes()
            embeddings.append(emb)

    if len(embeddings) < 3:
        print(f" Warning: {roll_no} has only {len(embeddings)} valid embeddings")
        while len(embeddings) < 3:
            embeddings.append(None)

    conn = sqlite3.connect("students.db")
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE students
        SET embedding1=?, embedding2=?, embedding3=?
        WHERE roll_no=?
    """, (embeddings[0], embeddings[1], embeddings[2], roll_no))
    conn.commit()
    conn.close()

    print(f" Embeddings updated for Roll No: {roll_no}")

In [12]:
update_student_embeddings("101", "sample_images/tushar_imgs")
update_student_embeddings("102", "sample_images/dhairya_imgs")
update_student_embeddings("103", "sample_images/debo_imgs")
update_student_embeddings("104", "sample_images/archit_imgs")

 Embeddings updated for Roll No: 101
 Embeddings updated for Roll No: 102
 Embeddings updated for Roll No: 103
 Embeddings updated for Roll No: 104
